Verifica a versão do python

Faz a instalação dos pacotes nas versões corretas

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers[torch]==4.41.2
!pip install datasets==2.20.0
!pip install huggingface_hub==0.23.4
!pip install ipywidgets==7.7.1
!pip install accelerate==0.27.2
!pip install nltk rouge-score
!pip install sentencepiece
!pip install evaluate
!pip install bert_score
!pip install openpyxl

Verifica se a placa de vídeo está disponível

Configura a forma de rodar - 
dev = usando o dataset de desenvolvimento com 7mil QA
prod = usando o dataset completo com cerca de 56mil QA

lang - escolher se utiliza o dataset de python ou java

In [1]:
run_mode = "dev"
#run_mode = "prod"
lang_mode = "Python"
#lang_mode = "Java"

#modelo = "flan-t5-base"
modelo = "codet5-base"
#modelo = "t5-base"

In [2]:
import warnings
# Suprime todos os warnings
warnings.filterwarnings("ignore")

Cria as pastas para salvar o modelo treinado

In [3]:
import os

if(run_mode=="prod"):
    os.chdir("2024_q-a_bases/"+lang_mode)
else:
    os.chdir("2024_q-a_bases/dev/"+lang_mode)


In [4]:
from datasets import load_dataset
import json

# Carregue o dataset do arquivo JSON
dev = os.path.join("dev","dataset_dev.json")
test = os.path.join("test","dataset_test.json")
dataset = load_dataset('json', data_files={'dev': dev, 'test': test})

In [5]:
os.chdir("results")
path_models = os.listdir()
os.chdir(path_models[-1])
os.chdir("results_model")

In [6]:
path_model = "modelo_treinado"
path_tokenizer = "tokenizer_treinamento"

Avaliação no Conjunto de Teste

Carregar e usar o modelo ajustado

In [7]:
import transformers
from transformers import T5Tokenizer
from transformers import RobertaTokenizer
from transformers import T5ForConditionalGeneration

# Carregue o modelo ajustado
model = T5ForConditionalGeneration.from_pretrained(path_model)
tokenizer = RobertaTokenizer.from_pretrained(path_tokenizer)

def gerar_resposta(context, question):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=150, num_return_sequences=1,
                             no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resposta


Gero e salvo respostas para cada uma das perguntas na base de testes

In [8]:
from tqdm import tqdm
perguntas = []
contextos = []
respostas = []
respostas_esperadas = []
i =0
with tqdm(total=len(dataset["test"])) as progressbar:
    for linha in dataset["test"]:
        respostas.append(gerar_resposta(linha["context"],linha["question"]))
        respostas_esperadas.append(linha["answer"])
        perguntas.append(linha["question"])
        contextos.append(linha["context"])
        i+=1
        progressbar.update(1)


chatbot = []
i =0
print("Geração concluída")



100%|██████████| 100/100 [00:22<00:00,  4.47it/s]

Geração concluída


In [9]:
for pergunta, contexto, resposta, respostas_esperada in zip(perguntas, contextos, respostas, respostas_esperadas):
    chatbot.append({
        "context": contexto,
        "question": pergunta,
        "answer": resposta,
        "exp_answer": respostas_esperada
    })
    i+=1

In [10]:
# Escreva os dados no arquivo JSON
with open('chatbot_test.json', 'w', encoding='utf-8') as f:
    json.dump(chatbot, f, ensure_ascii=False, indent=4)

print("Arquivo JSON test criado com sucesso.")

Arquivo JSON test criado com sucesso.


Avaliando

In [11]:
import numpy as np
from evaluate import load
from bert_score import BERTScorer
from nltk.translate.bleu_score import corpus_bleu

In [12]:

def compute_bleu(references, hypothesis):
    # https://stackoverflow.com/questions/73938010/which-bleu-smoothing-function-is-commonly-used-for-image-captioning-evaluation
    references = [[reference.split()] for reference in references]
    hypothesis = [hypothesis.split() for hypothesis in hypothesis]
    bleu_scores = corpus_bleu(references, hypothesis)
    return np.mean(bleu_scores)


def compute_meteor(reference, hypothesis):
    metric = load('meteor')
    results = metric.compute(predictions=hypothesis, references=reference)
    return results['meteor']


def compute_rouge(reference, hypothesis):
    metric = load('rouge')
    results = metric.compute(predictions=hypothesis, references=reference, use_stemmer=False)
    return results


def compute_bertscore(reference, hypothesis):
    scorer = BERTScorer(lang='pt', rescale_with_baseline=True)
    precision, recall, f1_score = scorer.score(hypothesis, reference, verbose=True)
    return {
        'precision': precision.mean().item(),
        'recall': recall.mean().item(),
        'f1_score': f1_score.mean().item()
    }


def compute_eval_metrics(all_references: list, all_predictions: list) -> dict:
    bleu_score = compute_bleu(all_references, all_predictions)
    meteor_score = compute_meteor(all_references, all_predictions)
    rouge_scores = compute_rouge(all_references, all_predictions)
    bert_scores = compute_bertscore(all_references, all_predictions)
    return {
        'rouge1': rouge_scores['rouge1'],
        'rouge2': rouge_scores['rouge2'],
        'rougeL': rouge_scores['rougeL'],
        'bert_score_f1': bert_scores['f1_score'],
        'bleu': bleu_score,
        'meteor': meteor_score
    }

In [13]:
resultado = compute_eval_metrics(respostas_esperadas,respostas)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lopes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lopes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lopes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 0.45 seconds, 223.13 sentences/sec


In [14]:
print(resultado["rouge1"])

0.5686702235672823


In [15]:
for key, value in resultado.items():
    print(f"{key}: {value}")

rouge1: 0.5686702235672823
rouge2: 0.03472524787040916
rougeL: 0.5600739521915992
bert_score_f1: 0.6182286739349365
bleu: 2.21076469358935e-155
meteor: 0.2873008942052908


In [16]:
# Escreva os dados no arquivo JSON
with open('chatbot_test_resultado_h.json', 'w', encoding='utf-8') as f:
    json.dump(resultado, f, ensure_ascii=False, indent=4)

print("Arquivo JSON test criado com sucesso.")
os.chdir("..")
os.chdir("..")
os.chdir("..")
os.chdir("..")
os.chdir("..")

Arquivo JSON test criado com sucesso.


Tabulação de todos os resultados ja colhidos

In [17]:
def read_data(text_file_path):
  # Carregar o arquivo de texto
  with open(text_file_path, 'r') as file:
    data = file.read()
  return data.split('\n')

In [18]:
def carregar_resultado_anterior(path):
    os.chdir(path)
    os.chdir("results_model")
    
    caminho_arquivo = "chatbot_test_resultado_h.json"

    # Abrir e carregar o arquivo JSON
    with open(caminho_arquivo, 'r') as arquivo:
        dados_resultados = json.load(arquivo)
    dados_basicos_resultado = read_data("info.txt")
    os.chdir("..")
    os.chdir("..")
    return [
            dados_basicos_resultado[0],dados_basicos_resultado[1],
            dados_basicos_resultado[2],dados_basicos_resultado[3],
            dados_basicos_resultado[4],
            dados_resultados['rouge1'],dados_resultados["rouge2"],
            dados_resultados["rougeL"],dados_resultados["bert_score_f1"],
            dados_resultados["bleu"],dados_resultados["meteor"]]

In [19]:
from datasets import load_dataset

def carregar_resultados(path):
    os.chdir(path)
    resultados = []

    os.chdir("results")
    resultados_paths = os.listdir()
    resultados = []
    for linha in resultados_paths:
        resultados.append(carregar_resultado_anterior(linha))
    os.chdir("..")
    os.chdir("..")
    return resultados
    


In [20]:
resultados_Python = carregar_resultados("Python")

In [21]:
resultados_Java = carregar_resultados("Python")

In [22]:
os.chdir("dev")
resultados_dev_python = carregar_resultados("Python")
resultados_dev_java = carregar_resultados("Java")
os.chdir("..")


In [23]:
import pandas as pd

def resultado_to_df(base):
    linguagem = []
    modo = []
    modelo = []
    tam_base = []
    rouge1 = []
    rouge2 = []
    rougeL= []
    bert = []
    bleu = []
    meteor = []
    for linha in base:
        linguagem.append(linha[0])
        modo.append(linha[1])
        modelo.append(linha[2])
        tam_base.append(linha[3])
        rouge1.append(linha[4])
        rouge2.append(linha[5])
        rougeL.append(linha[6])
        bert.append(linha[7])
        bleu.append(linha[8])
        meteor.append(linha[9])

        

    data = {
        'Language': linguagem,
        'Mode': modo,
        'Modelo': modelo,
        'tam_base': tam_base,
        'rouge1' : rouge1,
        'rouge2' : rouge2,
        'rougeL' : rougeL,
        'bert_score_f1': bert,
        'bleu': bleu,
        'meteor': meteor
    }
    return pd.DataFrame(data)

In [24]:
if(len(resultados_dev_python)>0): df = resultado_to_df(resultados_dev_python)
if(len(resultados_dev_java)>0): df2 = resultado_to_df(resultados_dev_java)
if(len(resultados_Python)>0): df3 = resultado_to_df(resultados_Python)
if(len(resultados_Java)>0): df4 = resultado_to_df(resultados_Java)

In [25]:
# Caminho para o arquivo Excel
caminho_arquivo = 'compilacao_resultados.xlsx'

# Exportar os DataFrames para diferentes abas em um único arquivo Excel
with pd.ExcelWriter(caminho_arquivo, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='dev_python', index=False)
    if(len(resultados_dev_java)>0): df2.to_excel(writer, sheet_name='dev_java', index=False)
    if(len(resultados_Python)>0): df3.to_excel(writer, sheet_name='Python', index=False)
    if(len(resultados_Java)>0): df4.to_excel(writer, sheet_name='Java', index=False)

print(f'Dados exportados para {caminho_arquivo} com sucesso!')


Dados exportados para compilacao_resultados.xlsx com sucesso!


In [26]:
os.chdir("..")